### Import libraries

In [1]:
from math import sqrt
from numpy import array
from numpy import mean
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from pandas.io.json import json_normalize
from sklearn.metrics import explained_variance_score,max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score

#models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D

from datetime import datetime

# Models

### LSTM

In [2]:
def lstm_model(cfg):
    model = Sequential()
    model.add(LSTM(cfg.get('n_nodes'), activation='relu', input_shape=(cfg.get('n_input'), cfg.get('n_features'))))
    model.add(Dense(cfg.get('n_nodes'), activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

### Conv1D

In [3]:
def conv1d_model(cfg):
    model = Sequential()
    model.add(Conv1D(filters=cfg.get('n_filters'), kernel_size=cfg.get('n_kernel'), activation='relu',
                     input_shape=(cfg.get('n_input'), cfg.get('n_features'))))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

## Configuration

In [4]:
def configuration(name, config):
    name = name.split("-")[0]
    value = config.get(name)
    if(name=="lstm"):
        n_input = value.get('n_input')
        n_nodes = value.get('n_nodes')
        n_epochs = value.get('n_epochs')
        n_batch = value.get('n_batch')
        n_diff = value.get('n_diff')
        n_features = value.get('n_features')
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_nodes:
                                cfg = { 'n_input': m,
                                        'n_nodes': n,
                                        'n_epochs': i,
                                        'n_batch': j,
                                        'n_diff': k,
                                        'n_features': l}
                                configs.append(cfg)
        return configs
    
    if(name=="conv1d"):
        n_input = value.get('n_input')
        n_filters = value.get('n_filters')
        n_kernel = value.get('n_kernel')
        n_epochs = value.get('n_epochs')
        n_batch = value.get('n_batch')
        n_diff = value.get('n_diff')
        n_features = value.get('n_features')
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_filters:
                                for o in n_kernel:
                                    cfg = {
                                        'n_input': m,
                                        'n_filters': n,
                                        'n_kernel': o,
                                        'n_epochs': i,
                                        'n_batch': j,
                                        'n_diff': k,
                                        'n_features': l}
                                    configs.append(cfg)
        return configs

In [5]:
def default_config():
    config = {'lstm':{
                'n_input':[12],
                'n_nodes':[100],
                'n_epochs':[2,3],
                'n_batch':[1],
                'n_diff':[12],
                'n_features':[1]},
        'conv1d':{
            'n_input':[12],
            'n_filters':[64],
            'n_kernel':[3],
            'n_epochs':[50],
            'n_batch':[1],
            'n_diff':[12],
            'n_features':[1]}
        }
    return config

### Data

In [6]:
series = read_csv('airline-passengers.csv', header=0, index_col=0)
data = series.values
n_test = 12

In [7]:
def train_test_split(data):
    return data[:int(0.8*data.shape[0])], data[int(0.8*data.shape[0]):]

### Metrics

In [8]:
def measure_error(actual, predicted):
    return {'EVC': explained_variance_score(actual, predicted),
            'ME': max_error(actual, predicted),
            'MAE': mean_absolute_error(actual, predicted),
            'MSE': mean_squared_error(actual, predicted),
            'SMSE': sqrt(mean_squared_error(actual, predicted)),
            'MSLE': mean_squared_log_error(actual, predicted),
            'MEDAE': median_absolute_error(actual, predicted),
            'R2': r2_score(actual, predicted)}

### Functions

In [9]:
def series_to_supervised(data, n_in=1, n_out=1):
    df = DataFrame(data)
    cols = list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))    
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    agg = concat(cols, axis=1)
    agg.dropna(inplace=True)
    return agg.values

In [10]:
def difference(data, order):
    return [data[i] - data[i - order] for i in range(order, len(data))]

In [11]:
def model_fit(train, config, create_model):
    if config.get('n_diff') > 0:
        train = difference(train, config.get('n_diff'))
    data = series_to_supervised(train, n_in=config.get('n_input'))
    train_x, train_y = data[:, :-1], data[:, -1]
    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], config.get('n_features')))

    model = create_model
    model.fit(train_x, train_y, epochs=config.get('n_epochs'), batch_size=config.get('n_batch'), verbose=0)
    return model

In [12]:
def model_predict(model, history, config, create_model):
    correction = 0.0
    if config.get('n_diff') > 0:
        correction = history[-config.get('n_diff')]
        history = difference(history, config.get('n_diff'))
    x_input = array(history[-config.get('n_input'):]).reshape((1, config.get('n_input'), 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return correction + yhat[0]

In [13]:
def walk_forward_validation(data, n_test, cfg, create_model,array_of_results):
    predictions = list()
    # split dataset
    train, test = train_test_split(data)
    # fit model
    model = model_fit(train, cfg, create_model)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg, create_model)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = list(measure_error(test, predictions).values())

    return error

In [14]:
def repeat_evaluate(data, config, n_test,create_model,array_of_results, n_repeats=2):
    #MOŻE TU BYĆ PROBLEM W JAK ZMIENIMY n_repeats
    scores = [walk_forward_validation(data, n_test, config, create_model,array_of_results) for _ in range(n_repeats)]
    results ={}
    scores = zip(scores[0],scores[1])
    result = []
    for s in scores:
        result.append(mean(s))
        

    results["Model"] = create_model.name
    results["EVC"] = result[0]
    results["ME"] = result[1]
    results["MAE"] = result[2]
    results["MSE"] = result[3]
    results["RMSE"] = result[4]
    results["MSLE"] = result[5]
    results["MED"] = result[6]
    results["R2"] = result[7]
  
    return results

In [15]:
def run():

    cfg_list = []
    cfg_list.append(lstm_model)
    cfg_list.append(conv1d_model)
    
    def_config_list = []
    def_config_list.append(configuration('lstm',default_config()))
    def_config_list.append(configuration('conv1d',default_config()))
    array_of_results = []
    results = {}
    array_of_models = []
    for cfg_model,def_cfg in zip(cfg_list,def_config_list):
        tmp_results = []
        i=1
        results.clear()
        for cfg in def_cfg:
            model = cfg_model(cfg)
            model._name=str(cfg_model.__name__.split("_")[0]+"-"+str(i))
            i +=1
            array_of_models.append(model)
            array_of_results.append(repeat_evaluate(data, cfg, n_test, model, array_of_results))
        
    return (pd.DataFrame(array_of_results), array_of_models)

In [16]:
def find_max(measure_statistics_df,stat):
    max =0
    model = ""
    for i in measure_statistics_df:
        if(max <= int(i.iloc[0][stat])):
            model = i.iloc[0]["Model"]
    return model

In [17]:
def get_model(list_of_models,model_name):
    index = -1;
    for m in list_of_models:
        index += 1
        if(model_name==m.name):
            return (m,index)

In [27]:
df,models = run()

In [33]:
def run_model(model_name, cfg,list_models, result_df):
    array_of_results = []
    i = int(model_name.split("-")[1])
    model = eval(model_name.split("-")[0]+"_model")(cfg)
    model._name=str(model_name.split("-")[0]+"-"+str(i+1))
    result = repeat_evaluate(data, cfg, n_test, model, array_of_results)
    result_x = pd.DataFrame.from_dict([result])
    list_models.append(model)
    return pd.concat([result_x,result_df]), list_models

In [35]:
def test():  
#     df_model = df[get_model(models,find_max(df,"R2"))[1]]
    df,_ = run_model(df.iloc[0]["Model"],{
                'n_input':12,
                'n_nodes':40,
                'n_epochs':3,
                'n_batch':2,
                'n_diff':12,
                'n_features':1},models,df)
    df, _ = run_model(df.iloc[0]["Model"],{
                'n_input':12,
                'n_nodes':40,
                'n_epochs':5,
                'n_batch':2,
                'n_diff':12,
                'n_features':1},models,df)
    return df

In [44]:
def get_config(list_of_models):
    list_of_configs = {}
    for m in list_of_models:
        list_of_configs[m.get_config()['name']] = m.get_config()['layers']
    return list_of_configs

In [45]:
def convert_config_to_datframes(config_json):

    index = -1
    tmp = []
    list_of_final_dataframes = []
    dfJson1 =None
    for i in config_json:
        if dfJson1 is not None:
            list_of_final_dataframes.append(dfJson1)
        dfJson1 = pd.DataFrame(index=[],columns=['name'])
        for j in config_json[i]:       
            index += 1
            dfJson1.loc[index,'date']=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            dfJson1.loc[index,'class_name']=j['class_name']
            dfJson1.loc[index,'name']=i
            for k in j['config']:
                dfJson1.loc[index,'cfg-'+str(k)]=str(j['config'][k]) 
    return list_of_final_dataframes

In [46]:
get_config(models)

{'lstm-1': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 12, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'lstm_3_input'}},
  {'class_name': 'LSTM',
   'config': {'name': 'lstm_3',
    'trainable': True,
    'batch_input_shape': (None, 12, 1),
    'dtype': 'float32',
    'return_sequences': False,
    'return_state': False,
    'go_backwards': False,
    'stateful': False,
    'unroll': False,
    'time_major': False,
    'units': 100,
    'activation': 'relu',
    'recurrent_activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'recurrent_initializer': {'class_name': 'Orthogonal',
     'config': {'gain': 1.0, 'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'unit_forget_bias': True,
    'kernel_regularizer': None,
    'recurrent_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer':

In [47]:
convert_config_to_datframes(get_config(models))[0].head(25)

,name,date,class_name,cfg-batch_input_shape,cfg-dtype,cfg-sparse,cfg-ragged,cfg-name,cfg-trainable,cfg-return_sequences,...,cfg-kernel_regularizer,cfg-recurrent_regularizer,cfg-bias_regularizer,cfg-activity_regularizer,cfg-kernel_constraint,cfg-recurrent_constraint,cfg-bias_constraint,cfg-dropout,cfg-recurrent_dropout,cfg-implementation
0,lstm-1,2020-11-11 00:56:20,InputLayer,"(None, 12, 1)",float32,False,False,lstm_3_input,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,lstm-1,2020-11-11 00:56:20,LSTM,"(None, 12, 1)",float32,NaN,NaN,lstm_3,True,False,...,None,None,None,None,None,None,None,0.0,0.0,2
2,lstm-1,2020-11-11 00:56:20,Dense,NaN,float32,NaN,NaN,dense_7,True,NaN,...,None,NaN,None,None,None,NaN,None,NaN,NaN,NaN
3,lstm-1,2020-11-11 00:56:20,Dense,NaN,float32,NaN,NaN,dense_8,True,NaN,...,None,NaN,None,None,None,NaN,None,NaN,NaN,NaN
